# **Amazon Reviews Sentiment Analysis**
---

### **Aim**:

The primary objective is to conduct sentiment analysis on Amazon product reviews. This involves taking a product link as input, extracting some product details, utilizing Google's Gemini-Pro AI model for sentiment analysis on the reviews, and presenting the insightful results in a user-friendly format.


### **Importing required libraries for scrapping**

In [41]:
from bs4 import BeautifulSoup
import requests

In [42]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
    "Accept-Language": "en-US, en;q=0.5",
}

URL = "https://www.amazon.in/Logitech-MK240-NANO-Mouse-Keyboard/dp/B01N4EV2TL/?_encoding=UTF8&pd_rd_w=iXhOQ&content-id=amzn1.sym.87579d35-fd9c-4df4-a4dd-ff69ee6fde5e%3Aamzn1.symc.acc592a4-4352-4855-9385-357337847763&pf_rd_p=87579d35-fd9c-4df4-a4dd-ff69ee6fde5e&pf_rd_r=F2VCS1EK4NVR0ZFW6Y0W&pd_rd_wg=QK4lQ&pd_rd_r=d66cee94-8b5f-4585-a691-6273ec46a42e&ref_=pd_gw_ci_mcx_mr_hp_d&th=1"

### **Getting page source**

In [43]:
webpage = requests.get(URL, headers=HEADERS)

### **Parsing using BS4**

In [44]:
soup = BeautifulSoup(webpage.content, "html.parser")

### **Extracting product details**

- Title
- Price
- Image
- Ratings

In [46]:
product_title = soup.find("span", attrs={"id": "productTitle"}).text.strip()

product_currency_symbol = soup.find(
    "span", attrs={"class": "a-price-symbol"}
).text.strip()

product_price_whole = soup.find("span", attrs={"class": "a-price-whole"}).text.strip()

product_price = f"{product_currency_symbol} {product_price_whole}"

product_image_url = soup.find("img", attrs={"id": "landingImage"})["src"]

product_rating = (
    soup.find("a", attrs={"class": "a-popover-trigger a-declarative"}).text.split()[0]
    + "stars"
)

product_ratings_count = (
    soup.find("span", attrs={"id": "acrCustomerReviewText"}).text.strip() + "Ratings"
)

### **Unique URL Construction for Reviews Pages**

Since the reviews page URL varies, with different patterns for paginated and non-paginated URLs, and updates dynamically when navigating through pages, a unique URL construction method has been devised.

**Key Considerations:**
- Handles paginated and non-paginated review page URLs.
- Dynamically updates for 'next' and 'prev' actions.
- Constructed URL allows for efficient scraping with page number parameterization.


In [47]:
product_review_link = (
    "https://www.amazon.in"
    + soup.find("a", attrs={"class": "a-link-emphasis a-text-bold"})["href"].split("?")[
        0
    ]
    + "/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber={}"
)

### **Printing details**

In [48]:
print(product_title)
print(product_price)
print(product_image_url)
print(product_rating)
print(product_ratings_count)
print(product_review_link.format(1))

Logitech MK240 Nano Wireless USB Keyboard and Mouse Set, 12 Function Keys 2.4GHz Wireless, 1000DPI, Spill-Resistant Design, PC/Mac, Black/Chartreuse Yellow
₹ 1,495.
https://m.media-amazon.com/images/I/41tLaG2nSpL._SX300_SY300_QL70_ML2_.jpg
4.3stars
9,219 ratingsRatings
https://www.amazon.in/Logitech-MK240-NANO-Mouse-Keyboard/product-reviews/B01N4EV2TL/ref=cm_cr_dp_d_show_all_btm/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1


### **Function to get reviews from a single page**

In [49]:
def get_reviews(review_url):
    review_webpage = requests.get(review_url, headers=HEADERS)
    review_soup = BeautifulSoup(review_webpage.content, "html.parser")
    reviews = review_soup.find_all(
        "span", attrs={"class": "a-size-base review-text review-text-content"}
    )
    return [review_text.text.strip() for review_text in reviews]

### **Getting reviews text in list for first page**

In [65]:

review_url = product_review_link.format(1)
reviews = get_reviews(review_url)

In [66]:
print("Total Reviews:", len(reviews))

Total Reviews: 10


### **Importing Required libraries for analysis**

In [67]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()

True

In [68]:
api_key = os.getenv("GENAI_API_KEY")
genai.configure(api_key=api_key)

### **Initialising Model**

In [69]:
generation_config = {
    "temperature": 0.9,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 2048,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
]

model = genai.GenerativeModel(
    model_name="gemini-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

### **Constructing Prompt**

In [73]:
prompt = "Please analyse the sentiment of users who purchased the product and gave these reviews. Give detailed summary, pros and cons about product."
prompt += "\n\nReviews:\n"

for i, review in enumerate(reviews, start=1):
    prompt += f"Review {i}: {review}\n"

### **Generating sentiment of reviews**

In [ ]:
response = model.generate_content([prompt])

### **Display reposnse as markdown**

In [74]:
from IPython.display import display, Markdown

### **Results**

In [75]:
display(Markdown(response.text))

Overall Sentiment:
The overall sentiment of the users who purchased the product and gave these reviews is positive. Many users have expressed satisfaction with the product's ease of use, comfort, and functionality. However, there are some minor issues mentioned by a few users, such as the color combination and the absence of a caps lock indicator.

Pros:
1. Easy to connect and use.
2. Comfortable keys and mouse clicks.
3. Good ergonomics design.
4. Good battery life.
5. Keyboard and mouse quality is good.
6. Sleek design.
7. Good quality mouse with a place to keep the USB connector inside.
8. Compact design.
9. Fast response.

Cons:
1. Color combination is dull.
2. No hand rest extension on the keyboard.
3. Stand cushions are not sticky and often fall off.
4. Absence of a caps lock indicator.
5. Keys are pressed very easily by minimal pressure.
6. Mouse movement slowed down and stopped working after 8 months.

Summary:
The Logitech wireless keyboard and mouse combo is a popular choice among users due to its ease of use, comfort, and functionality. The keyboard features comfortable keys with a pleasant sound, while the mouse provides good clicks and smooth operation. The combo is easy to connect and use, and the battery life is decent. However, some users have mentioned issues such as the color combination, the lack of a hand rest extension on the keyboard, and the non-sticky stand cushions. Additionally, one user reported that the mouse movement slowed down and stopped working after 8 months of use. Overall, this product is a good choice for users looking for a reliable and affordable wireless keyboard and mouse combo, but they should be aware of the potential issues mentioned by other users.

----
### **Conclusion**

In this notebook, we successfully developed an Amazon Sentiment Analyzer empowered by Google's Gemini-Pro AI model. The tool accepts an Amazon product link as input, extracts some details from the product page, and employs the Gemini-Pro AI model to perform sentiment analysis on customer reviews.

**Key Achievements:**
1. **Product Details:** Succesfully extracted some product details like title, price, ratings and reviews.

2. **Dynamic URL Construction:** Overcame challenges posed by varying review page URLs by implementing a method to construct a unique URL, accommodating both paginated and non-paginated structures.
   
3. **Gemini-Pro AI Integration:** Leveraged Google's Gemini-Pro AI model for sentiment analysis, enhancing the accuracy and depth of our review evaluations.

Thank you for taking the time to explore this notebook! If you have any questions or suggestions for improvement, please feel free to reach out.

**Happy coding!**

----
### **Disclaimer**

This Amazon Sentiment Analyzer is created for educational purposes and personal learning. It is done to explore web scraping and test the capabilities of Google's Gemini AI model. 

**Important Points:**

- **Learning Project:** This tool is developed solely for educational and learning purposes.

- **No Commercial Use:** It is not intended for any commercial or business applications.

By using this tool, you acknowledge its educational nature and agree to use it responsibly. The developers are not liable for any misuse or consequences arising from its use.
